In [8]:
!pip install -U vnstock


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [9]:
import pandas as pd
from vnstock import Quote
from datetime import datetime, timedelta
import os
import time

print("Import thư viện thành công!")

Import thư viện thành công!


In [10]:
from vnstock import Listing

# Lấy danh sách mã VN30
try:
    symbols = Listing().symbols_by_group("VN30")
    if isinstance(symbols, pd.Series):
        symbols = symbols.tolist()  # Chuyển đổi Series thành list nếu cần
    print(f"✅ Lấy thành công {len(symbols)} mã VN30: {symbols}")
except Exception as e:
    print(f"❌ Lỗi khi lấy danh sách mã VN30: {e}")
    symbols = []  # Fallback nếu không lấy được mã

✅ Lấy thành công 30 mã VN30: ['ACB', 'BCM', 'BID', 'CTG', 'DGC', 'FPT', 'GAS', 'GVR', 'HDB', 'HPG', 'LPB', 'MBB', 'MSN', 'MWG', 'PLX', 'SAB', 'SHB', 'SSB', 'SSI', 'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIB', 'VIC', 'VJC', 'VNM', 'VPB', 'VRE']


In [11]:
def crawl_stock_data(symbol, start_date, end_date, interval='1d', max_retries=3):
    """
    Crawl dữ liệu lịch sử cho một mã cổ phiếu với retry mechanism
    
    Args:
        symbol: Mã cổ phiếu (VD: 'FPT', 'VNM')
        start_date: Ngày bắt đầu (format: 'YYYY-MM-DD')
        end_date: Ngày kết thúc (format: 'YYYY-MM-DD')
        interval: Khung thời gian ('1m', '5m', '15m', '30m', '1h', '1d')
        max_retries: Số lần retry tối đa
    
    Returns:
        DataFrame hoặc None nếu lỗi
    """
    for attempt in range(max_retries):
        try:
            quote = Quote(symbol=symbol)
            data = quote.history(start=start_date, end=end_date, interval=interval)
            
            if data is not None and not data.empty:
                return data
            else:
                return None
                
        except Exception as e:
            error_msg = str(e)
            
            # Nếu bị rate limit, đợi lâu hơn
            if 'rate' in error_msg.lower() or 'limit' in error_msg.lower() or 'retry' in error_msg.lower():
                wait_time = (attempt + 1) * 5  # 5s, 10s, 15s
                print(f"  ⏳ Rate limit! Đợi {wait_time}s... (attempt {attempt+1}/{max_retries})")
                time.sleep(wait_time)
            else:
                if attempt == max_retries - 1:
                    print(f"  ❌ Lỗi {symbol}: {error_msg[:100]}")
                return None
    
    return None

print("Đã định nghĩa function crawl_stock_data với retry mechanism")

Đã định nghĩa function crawl_stock_data với retry mechanism


In [12]:
# Tạo folder để lưu CSV (VN30, 480 ngày)
output_folder = 'stock_data_vn30_480days'
os.makedirs(output_folder, exist_ok=True)

# Thiết lập thời gian crawl (480 ngày gần nhất)
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=2000)).strftime('%Y-%m-%d')

print(f"📁 Folder output: {output_folder}")
print(f"📅 Khoảng thời gian: {start_date} đến {end_date} (480 ngày)")
print(f"⏱️  Interval: 1 ngày (1d)")

📁 Folder output: stock_data_vn30_480days
📅 Khoảng thời gian: 2020-06-09 đến 2025-11-30 (480 ngày)
⏱️  Interval: 1 ngày (1d)


In [13]:
# Crawl dữ liệu cho tất cả các mã (VN30, tối đa 30 mã, 480 ngày)
success_count = 0
fail_count = 0
fail_symbols = []

print("🚀 BẮT ĐẦU CRAWL DỮ LIỆU VN30 (480 ngày)...")
print("="*60)

for i, symbol in enumerate(symbols, 1):
    print(f"[{i}/{len(symbols)}] Crawling {symbol}...", end=' ')
    
    # Crawl dữ liệu với interval 1 ngày
    data = crawl_stock_data(symbol, start_date, end_date, interval='1d')
    
    if data is not None and not data.empty:
        # Tạo tên file CSV (480 ngày)
        filename = f"{output_folder}/{symbol}_480days_daily.csv"
        
        # Xuất ra CSV
        data.to_csv(filename, index=True, encoding='utf-8-sig')
        
        print(f"✅ {len(data)} rows → {filename}")
        success_count += 1
    else:
        print(f"❌ Không có dữ liệu")
        fail_count += 1
        fail_symbols.append(symbol)
    
    # Delay dài hơn để tránh rate limit (2-3 giây mỗi request)
    time.sleep(2)
    
    # Hiển thị tiến độ sau mỗi 10 mã
    if i % 10 == 0:
        print(f"  ⏳ Tiến độ: {i}/{len(symbols)} - Thành công: {success_count}, Thất bại: {fail_count}")
        # Nghỉ thêm sau mỗi 10 mã
        time.sleep(3)

print("\n" + "="*60)
print("🏁 HOÀN THÀNH!")
print(f"✅ Thành công: {success_count}/{len(symbols)}")
print(f"❌ Thất bại: {fail_count}/{len(symbols)}")

if fail_symbols:
    print(f"\n❌ Các mã thất bại: {', '.join(fail_symbols[:20])}")
    if len(fail_symbols) > 20:
        print(f"   ... và {len(fail_symbols) - 20} mã khác")

🚀 BẮT ĐẦU CRAWL DỮ LIỆU VN30 (480 ngày)...
[1/30] Crawling ACB... ✅ 1431 rows → stock_data_vn30_480days/ACB_480days_daily.csv
[2/30] Crawling BCM... ✅ 1431 rows → stock_data_vn30_480days/BCM_480days_daily.csv
[3/30] Crawling BID... ✅ 1431 rows → stock_data_vn30_480days/BID_480days_daily.csv
[4/30] Crawling CTG... ✅ 1431 rows → stock_data_vn30_480days/CTG_480days_daily.csv
[5/30] Crawling DGC... ✅ 1431 rows → stock_data_vn30_480days/DGC_480days_daily.csv
[6/30] Crawling FPT... ✅ 1431 rows → stock_data_vn30_480days/FPT_480days_daily.csv
[7/30] Crawling GAS... ✅ 1431 rows → stock_data_vn30_480days/GAS_480days_daily.csv
[8/30] Crawling GVR... ✅ 1431 rows → stock_data_vn30_480days/GVR_480days_daily.csv
[9/30] Crawling HDB... ✅ 1431 rows → stock_data_vn30_480days/HDB_480days_daily.csv
[10/30] Crawling HPG... ✅ 1431 rows → stock_data_vn30_480days/HPG_480days_daily.csv
  ⏳ Tiến độ: 10/30 - Thành công: 10, Thất bại: 0
[11/30] Crawling LPB... ✅ 1431 rows → stock_data_vn30_480days/LPB_480days_dai

In [14]:
# Kiểm tra một số file đã tạo
print("📂 CÁC FILE ĐÃ TẠO (10 file đầu tiên):")
print("="*60)

csv_files = [f for f in os.listdir(output_folder) if f.endswith('.csv')]
for i, filename in enumerate(sorted(csv_files)[:10], 1):
    filepath = os.path.join(output_folder, filename)
    file_size = os.path.getsize(filepath)
    
    # Đọc file để kiểm tra số dòng
    df = pd.read_csv(filepath)
    
    print(f"{i}. {filename}")
    print(f"   📊 Số dòng: {len(df)}, Kích thước: {file_size:,} bytes")
    print(f"   📅 Cột: {list(df.columns)[:5]}")
    print()

print(f"Tổng số file CSV: {len(csv_files)}")
print(f"📁 Thư mục: {os.path.abspath(output_folder)}")

📂 CÁC FILE ĐÃ TẠO (10 file đầu tiên):
1. ACB_480days_daily.csv
   📊 Số dòng: 1431, Kích thước: 66,381 bytes
   📅 Cột: ['Unnamed: 0', 'time', 'open', 'high', 'low']

2. BCM_480days_daily.csv
   📊 Số dòng: 1431, Kích thước: 64,269 bytes
   📅 Cột: ['Unnamed: 0', 'time', 'open', 'high', 'low']

3. BID_480days_daily.csv
   📊 Số dòng: 1431, Kích thước: 66,662 bytes
   📅 Cột: ['Unnamed: 0', 'time', 'open', 'high', 'low']

4. CTG_480days_daily.csv
   📊 Số dòng: 1431, Kích thước: 67,288 bytes
   📅 Cột: ['Unnamed: 0', 'time', 'open', 'high', 'low']

5. DGC_480days_daily.csv
   📊 Số dòng: 1431, Kích thước: 66,845 bytes
   📅 Cột: ['Unnamed: 0', 'time', 'open', 'high', 'low']

6. FPT_480days_daily.csv
   📊 Số dòng: 1431, Kích thước: 67,844 bytes
   📅 Cột: ['Unnamed: 0', 'time', 'open', 'high', 'low']

7. GAS_480days_daily.csv
   📊 Số dòng: 1431, Kích thước: 65,793 bytes
   📅 Cột: ['Unnamed: 0', 'time', 'open', 'high', 'low']

8. GVR_480days_daily.csv
   📊 Số dòng: 1431, Kích thước: 66,746 bytes
   